This will use the ESA SNAP GPT tool to create GRD products from the already downloaded SLC imagery. With the GRD data, the Normalized Difference Polarization Index, VV/VH, VH/VV, Normalized VH index, and Normalized VV index will be used to create water masks. Otsu thresholding will be done to label pixels as subaqeuous or subaerial.

These labeled pixels will be used for two things:
1. Generating a water mask for the InSAR time series (maybe the lowest water extent)
2. Generating a time series of subaerial change for individual creation sites

Plan is to couple the InSAR and Area time series to estimate volumetric changes of wetland BUDM sites 

In [ ]:
import os
import subprocess
from datetime import datetime
import shutil
from osgeo import gdal
import numpy as np

# Functions

In [ ]:
def gpt_help():

    cmd = f'{GPT_PATH} -h -c {MEMORY_SIZE}'

    try:
        result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

def help_cmd(cmdname):

    cmd = f'{GPT_PATH} {cmdname} -h -c {MEMORY_SIZE}'

    try:
        result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

def project_dir(slcpath):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data for ISCE2 and mintpy
    work-dir = str
        path to the directory created in 01_get_slc.ipynb
    """

    # path for GRD data storage
    grd_dir = slcpath.replace('SLC', 'GRD')
    os.makedirs(grd_dir, exist_ok=True)

    # list of grd directories
    grdpaths = []
    for step in ['0_tnr', '1_cal', '2_deburst', '3_multilook', '4_speckle', '5_spgr', '6_grd', '7_subset']:
        grdpaths.append(os.path.join(grd_dir, step))
        os.makedirs(os.path.join(grd_dir, step), exist_ok=True)


    return grd_dir, grdpaths

# Establish GPT path, number of processors, and memory usage

In [ ]:
GPT_PATH = '/home/wcc/esa-snap/bin/gpt'
NUM_PROCESSORS = 24
MEMORY_SIZE = '96G'

# Get GRD images that were downloaded

In [ ]:
# assuming you have downloaded .zip files covering your AOI from ASF Vertex
# enter the file directory below
slc_zips = '/home/wcc/Desktop/SabineRS/Sentinel-1/SLC/ASCENDING/136/93'

slc_zips_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
slc_zips_dirs = [os.path.join(slc_zips, slc) for slc in slc_zips_list]
slc_zips_dates = [slc[17:25] for slc in slc_zips_list]

# Establish GRD working directories

In [ ]:
grd_dir, grdpaths = project_dir(slc_zips)

# Processing steps for SLC to GRD:
1. Thermal Noise Removal
2. Calibration
3. TOPSAR-Deburst
4. Multilook
5. Speckle-Filter
6. Range-Doppler Terrain Correciton to 20m resolution
7. Subset
8. Write to tif

In [ ]:
POLARISATIONS = 'VV,VH' #can define this here for the rest of the notebook, if need to change will do 

1. ThermalNoiseRemoval

In [ ]:
OUTPUT_NOISE = False
REINTRODUCE_TN = False
REMOVE_TN = True

tnr_outpath = os.path.join(grdpaths[0], f'{slc_zips_dates[0]}.dim')
tnr_cmd = f'{GPT_PATH} ThermalNoiseRemoval -SsourceProduct={os.path.join(slc_zips,slc_zips_list[0])} -PoutputNoise={OUTPUT_NOISE} -PreIntroduceThermalNoise={REINTRODUCE_TN} -PremoveThermalNoise={REMOVE_TN} -PselectedPolarisations={POLARISATIONS} -t {tnr_outpath} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
try:
    result = subprocess.run(tnr_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")


# tnr_outpaths = []
# for i, file in enumerate(slc_zips_list):
#     tnr_outpaths.append(os.path.join(grdpaths[0], f'{slc_zips_dates[i]}.dim'))
#     tnr_cmd = f'{GPT_PATH} ThermalNoiseRemoval -SsourceProduct={os.path.join(slc_zips,file)} -PoutputNoise={OUTPUT_NOISE} -PreIntroduceThermalNoise={REINTRODUCE_TN} -PremoveThermalNoise={REMOVE_TN} -PselectedPolarisations={POLARISATIONS} -t {tnr_outpaths[i]} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
#     try:
#         result = subprocess.run(tnr_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         print(result.stdout.decode())
#     except subprocess.CalledProcessError as e:
#         print(f"Command failed: {e.stderr.decode()}")

2. Calibration

In [ ]:
BETA_BAND = False
OUT_BETA = False
GAMMA_BAND = False
OUT_GAMMA = False
COMPLEX_IM = False
DB_IM = False
SIGMA_BAND = True

cal_outpath = os.path.join(grdpaths[1], f'{slc_zips_dates[0]}.dim')

tnr_cmd = f'{GPT_PATH} Calibration -Ssource={os.path.join(slc_zips,tnr_outpath)} -PcreateBetaBand={BETA_BAND} -PcreateGammaBand={GAMMA_BAND} -PoutputBetaBand={OUT_BETA} -PoutputGammaBand={OUT_GAMMA} -PoutputImageInComplex={COMPLEX_IM} -PoutputImageScaleInDb={DB_IM} -PoutputSigmaBand={SIGMA_BAND} -PselectedPolarisations={POLARISATIONS} -t {cal_outpath} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
try:
    result = subprocess.run(tnr_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")

# cal_outpaths = []
# for i, file in enumerate(slc_zips_list):
#     cal_outpaths.append(os.path.join(grdpaths[1], f'{slc_zips_dates[i]}.dim'))

#     tnr_cmd = f'{GPT_PATH} Calibration -Ssource={os.path.join(slc_zips,file)} -PcreateBetaBand={BETA_BAND} -PcreateGammaBand={GAMMA_BAND} -PoutputBetaBand={OUT_BETA} -PoutputGammaBand={OUT_GAMMA} -PoutputImageInComplex={COMPLEX_IM} -PoutputImageScaleInDb={DB_IM} -PoutputSigmaBand={SIGMA_BAND} -PselectedPolarisations={POLARISATIONS} -t {cal_outpaths} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
#     try:
#         result = subprocess.run(tnr_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         print(result.stdout.decode())
#     except subprocess.CalledProcessError as e:
#         print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(grdpaths[0])

3. TOPSAR-Deburst

In [ ]:
deburst_outpath = os.path.join(grdpaths[2], f'{slc_zips_dates[0]}.dim')

deburst_cmd = f'{GPT_PATH} TOPSAR-Deburst -Ssource={os.path.join(slc_zips,cal_outpath)} -PselectedPolarisations={POLARISATIONS} -t {deburst_outpath} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
try:
    result = subprocess.run(deburst_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")

# deburst_outpaths = []
# for i, file in enumerate(slc_zips_list):
#     deburst_outpaths.append(os.path.join(grdpaths[2], f'{slc_zips_dates[i]}.dim'))

#     deburst_cmd = f'{GPT_PATH} TOPSAR-Deburst -Ssource={os.path.join(slc_zips,file)} -PselectedPolarisations={POLARISATIONS} -t {deburst_outpaths} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
#     try:
#         result = subprocess.run(deburst_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         print(result.stdout.decode())
#     except subprocess.CalledProcessError as e:
#         print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(grdpaths[1])

4. Multilook

In [ ]:
SQUARE_PIXEL = False
AZI_LOOKS = '1'         # sentinel-1 is 22m in azimuth
RANGE_LOOKS = '5'       # sentinel-1 is between 2.7 and 3.5m in range, depending on topography

mlook_outpath = os.path.join(grdpaths[3], f'{slc_zips_dates[i]}.dim')

mlook_cmd = f'{GPT_PATH} Multilook -Ssource={os.path.join(slc_zips,deburst_outpath)} -PgrSquarePixel={SQUARE_PIXEL} -PnAzLooks={AZI_LOOKS} -PnRgLooks={RANGE_LOOKS} -t {mlook_outpath} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
try:
    result = subprocess.run(mlook_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")


# mlook_outpaths = []
# for i, file in enumerate(slc_zips_list):
#     mlook_outpaths.append(os.path.join(grdpaths[3], f'{slc_zips_dates[i]}.dim'))

#     mlook_cmd = f'{GPT_PATH} Multilook -Ssource={os.path.join(slc_zips,file)} -PgrSquarePixel={SQUARE_PIXEL} -PnAzLooks={AZI_LOOKS} -PnRgLooks={RANGE_LOOKS} -t {mlook_outpaths} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
#     try:
#         result = subprocess.run(mlook_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         print(result.stdout.decode())
#     except subprocess.CalledProcessError as e:
#         print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(grdpaths[2])

5. Speckle-Filter

In [ ]:
NEIGHBORHOOD_SIZE = 100 # integer, (1, 200]
DAMPING_FACTOR = 2      # integer, for the Frost filter only
NUM_LOOKS = 1.0         # integer, (0, *)
ESTIMATE_ENL = False
FILTER = 'Lee Sigma'  # str, otptions include None, Boxcar, Median, Frost, Gamma Map, Lee, Refined Lee, Lee Sigma, or IDAN
FILTER_XSIZE = 3        # integer, (1, 100]
FITLER_YSIZE = 3        # integer, (1, 100]
NUM_LOOKS_STR = '1'         # str of integer, options are '1', '2', '3', '4'
SIGMA_STR = '0.9'       # str, options are '0.5', '0.6', '0.7', '0.8', '0.9'
TARGET_WINDOW = '3x3'   # str, can be '3x3' or '5x5'
WINDOW_SIZE = '5x5'     # str, can be '5x5', '7x7', '9x9', '11x11', '13x13', or '15x15'  

filter_outpath = os.path.join(grdpaths[4], f'{slc_zips_dates[0]}.dim')

filter_cmd = f'{GPT_PATH} Speckle-Filter -Ssource={os.path.join(slc_zips,mlook_outpath)} -PanSize={NEIGHBORHOOD_SIZE} -PdampingFactor={DAMPING_FACTOR} -Penl={NUM_LOOKS} -PestimateENL={ESTIMATE_ENL} -Pfilter="{FILTER}" -PfilterSizeX={FILTER_XSIZE} -PfilterSizeY={FITLER_YSIZE} -PnumLooksStr={NUM_LOOKS_STR} -PsigmaStr={SIGMA_STR} -PtargetWindowSizeStr={TARGET_WINDOW} -PwindowSize={WINDOW_SIZE} -t {filter_outpath} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
try:
    result = subprocess.run(filter_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")


# filter_outpaths = []
# for i, file in enumerate(slc_zips_list):
#     filter_outpaths.append(os.path.join(grdpaths[4], f'{slc_zips_dates[i]}.dim'))

#     filter_cmd = f'{GPT_PATH} Speckle-Filter -Ssource={os.path.join(slc_zips,file)} -PanSize={NEIGHBORHOOD_SIZE} -PdampingFactor={DAMPING_FACTOR} -Penl={NUM_LOOKS} -PestimateENL={ESTIMATE_ENL} -Pfilter={FILTER} -PfilterSizeX={FILTER_XSIZE} -PfilterSizeY={FITLER_YSIZE} -PnumLooksStr={NUM_LOOKS_STR} -PsigmaStr={SIGMA_STR} -PtargetWindowSizeStr={TARGET_WINDOW} -PwindowSize={WINDOW_SIZE} -t {filter_outpaths} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
#     try:
#         result = subprocess.run(filter_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         print(result.stdout.decode())
#     except subprocess.CalledProcessError as e:
#         print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(grdpaths[3])

6. SRGR

In [ ]:
INTERP_METHOD = 'Sinc'      # str, options are 'Linear', 'Cubic', 'Cubic2', and 'Sinc'
WARP_POLY_PRDER = 4         # integer, options include 1, 2, 3, 4

srgr_outpath = os.path.join(grdpaths[5], f'{slc_zips_dates[0]}.dim')

srgr_cmd = f'{GPT_PATH} SRGR -Ssource={os.path.join(slc_zips,filter_outpath)} -PinterpolationMethod="{INTERP_METHOD} interpolation" -PwarpPolynomialOrder={WARP_POLY_PRDER} -t {srgr_outpath} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
try:
    result = subprocess.run(srgr_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")

# srgr_outpaths = []
# for i, file in enumerate(slc_zips_list):
#     srgr_outpaths.append(os.path.join(grdpaths[5], f'{slc_zips_dates[i]}.dim'))

#     srgr_cmd = f'{GPT_PATH} SRGR -Ssource={os.path.join(slc_zips,file)} -PinterploationMethod={INTERP_METHOD} -PwarpPolynomialOrder={WARP_POLY_PRDER} -t {srgr_outpaths} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
#     try:
#         result = subprocess.run(srgr_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         print(result.stdout.decode())
#     except subprocess.CalledProcessError as e:
#         print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(grdpaths[4])

7. GRD-Post

In [ ]:
grd_outpath = os.path.join(grdpaths[6], f'{slc_zips_dates[0]}.dim')

grd_cmd = f'{GPT_PATH} GRD-Post -Ssource={os.path.join(slc_zips,srgr_outpath)} -t {grd_outpath} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
try:
    result = subprocess.run(grd_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")

# grd_outpaths = []
# for i, file in enumerate(slc_zips_list):
#     grd_outpaths.append(os.path.join(grdpaths[5], f'{slc_zips_dates[i]}.dim'))

#     grd_cmd = f'{GPT_PATH} GRD-Post -Ssource={os.path.join(slc_zips,file)} -PinterploationMethod={INTERP_METHOD} -PwarpPolynomialOrder={WARP_POLY_PRDER} -t {grd_outpaths} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
#     try:
#         result = subprocess.run(grd_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         print(result.stdout.decode())
#     except subprocess.CalledProcessError as e:
#         print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(grdpaths[5])

8. Subset? (using the aoi from the initial wrapped ifg)

In [ ]:
# first, get the wrapped ifg from 02a_isce2_processing.ipynb

ds = gdal.Open('/home/wcc/Desktop/SabineRS/Sentinel-1/InSAR/interferometry/work/merged/filt_topophase.flat.geo.vrt', gdal.GA_ReadOnly)
slc = ds.GetRasterBand(1).ReadAsArray()
transform = ds.GetGeoTransform()
ds = None

# getting the min max of the axes
firstx = transform[0]
firsty = transform[3]
deltay = transform[5]
deltax = transform[1]
lastx = firstx+slc.shape[1]*deltax
lasty = firsty+slc.shape[0]*deltay
ymin = np.min([lasty,firsty])
ymax = np.max([lasty,firsty])
xmin = np.min([lastx,firstx])
xmax = np.max([lastx,firstx])

aoi = f'POLYGON(({round(xmax,2)} {round(ymin,2)}, {round(xmax,2)} {round(ymax,2)}, {round(xmin,2)} {round(ymax,2)}, {round(xmin,2)} {round(ymin,2)}, {round(xmax,2)} {round(ymin,2)}))'
print(aoi)

In [ ]:
COPY_METADATA = True
FULL_SWATH = False
REFERNCE_BAND = 'Sigma0_VV'

subset_outpath = os.path.join(grdpaths[7], f'{slc_zips_dates[0]}.dim')

subset_cmd = f'{GPT_PATH} Subset -Ssource={os.path.join(slc_zips,grd_outpath)} -PcopyMetadata={COPY_METADATA} -PfullSwath={FULL_SWATH} -PgeoRegion="{aoi}" -PreferenceBand={REFERNCE_BAND} -t {subset_outpath} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
try:
    result = subprocess.run(subset_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")

# subset_outpaths = []
# for i, file in enumerate(slc_zips_list):
#     subset_outpaths.append(os.path.join(subsetpaths[5], f'{slc_zips_dates[i]}.dim'))

#     subset_cmd = f'{GPT_PATH} subset-Post -Ssource={os.path.join(slc_zips,file)} -PinterploationMethod={INTERP_METHOD} -PwarpPolynomialOrder={WARP_POLY_PRDER} -t {subset_outpaths} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
#     try:
#         result = subprocess.run(subset_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#         print(result.stdout.decode())
#     except subprocess.CalledProcessError as e:
#         print(f"Command failed: {e.stderr.decode()}")

shutil.rmtree(grdpaths[6])

8. Write

In [ ]:
gpt_help()

In [ ]:
help_cmd('BandSelect')

In [ ]:
subset_outpath

In [ ]:
CLEAR_CACHE = False
DELETE_ON_FAILURE = True
OUTFILE = os.path.join(grd_dir, f'{slc_zips_dates[0]}.tif')
FILE_FORMAT = 'GeoTIFF / BigTIFF'         # options include BEAM-DIMAP, HDF5, and more
WRITE_ENTIRE_ROW = False

tiff_cmd = f'{GPT_PATH} Write -Ssource={os.path.join(slc_zips,subset_outpath)} -PclearCacheAfterRowWrite={CLEAR_CACHE} -PdeleteOutputOnFailure={DELETE_ON_FAILURE} -PwriteEntireTileRows={WRITE_ENTIRE_ROW} -t {OUTFILE} -f "{FILE_FORMAT}" -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
try:
    result = subprocess.run(tiff_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")

# shutil.rmtree(grdpaths[7])

In [ ]:
os.path.join(grd_dir, f'{slc_zips_dates[0]}.tif')

In [ ]:
test_cmd = f'{GPT_PATH} /home/wcc/Desktop/SabineRS/Sentinel-1/GRD/export_geotiff.xml -Pinput={subset_outpath} -Poutput={os.path.join(grd_dir, f"{slc_zips_dates[0]}.tif")}'
try:
    result = subprocess.run(test_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")